# Data Understanding
## StationID 
    100t คือ สถานีอุตุนิยมวิทยาจันทบุรี ต.วัดใหม่ อ.เมือง, จันทบุรี
- PM25 PM2.5 คือ ฝุ่นละอองขนาดเล็กไม่เกิน 2.5 ไมครอน
- PM10 ฝุ่นหยาบ ที่มีอนุภาคเส้นผ่านศูนย์กลาง 2.5–10ไมครอน
- O3 ก๊าซโอโซน
- CO โคบอลต์
- NO2 ก๊าซไนโตรเจนไดออกไซด์
- SO2 ซัลเฟอร์ไดออกไซด์
- WS ความเร็วลม
- WD ทิศทางลม
- TEMP อุณหภูมิ
- RH ควมชื้นสัมพัทธ์
- BP ความกดอากาศ
- RAIN ปริมาณน้ำฝน

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [4]:
# Load Datasets
df = pd.read_csv('data/PM25_Station100t_with_Prep.csv')
df

,DATETIMEDATA,PM25,PM10,O3,CO,NO2,SO2,WS,WD,TEMP,RH,BP,RAIN
0,2023-08-01 00:00:00,11,32,9.0,0.31,7.000000,1.000000,1.4,183,28.3,87,750,0.0
1,2023-08-01 01:00:00,12,35,9.0,0.22,7.410755,1.067401,1.3,186,28.0,88,749,0.0
2,2023-08-01 02:00:00,15,35,10.0,0.19,6.000000,1.000000,1.3,185,28.1,88,749,0.0
3,2023-08-01 03:00:00,15,31,9.0,0.19,5.000000,1.000000,1.1,181,27.8,88,749,0.0
4,2023-08-01 04:00:00,12,33,9.0,0.17,4.000000,1.000000,1.0,192,27.9,88,749,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1901,2023-10-19 05:00:00,21,30,3.0,0.36,5.000000,1.067401,0.2,354,25.0,95,751,0.0
1902,2023-10-19 06:00:00,15,36,3.0,0.38,4.000000,1.067401,0.1,12,25.1,94,752,0.0
1903,2023-10-19 07:00:00,13,31,6.0,0.39,5.000000,1.067401,0.4,9,26.5,86,752,0.0
1904,2023-10-19 08:00:00,23,43,12.0,0.42,7.000000,1.067401,0.4,354,28.6,77,752,0.0
